In [2]:
# AutoEncoder를 이용한 잡음 제거: REDNet 구조를 예시로 활용
# https://dschloe.github.io/python/tensorflow2.0/ch9_4_super_resolution/ 에서 발췌

import numpy as np                  # 행렬 연산을 위한 numpy 라이브러리
import matplotlib.pyplot as plt     # 시각화를 위한 matplotlib 라이브러리
# tensorflow와 tf.keras를 임포트합니다
import tensorflow as tf
from tensorflow import keras
# 사용 가능한 GPU 갯수를 체크 (GPU 사용시 정상 동작 확인 = 1)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.__version__) # tensorflow 버전 확인

Num GPUs Available:  0
2.16.1


In [3]:
fashion_mnist = keras.datasets.fashion_mnist    # fashion MNIST 데이터셋
(train_clean, _), (test_clean, _) = fashion_mnist.load_data()   # 데이터셋 불러오기
_, img_height, img_width = train_clean.shape   # 영상의 크기 받아오기

train_clean = train_clean.reshape(-1, img_height, img_width, 1)/255.   # batch 차원을 추가해서 학습 data를 변형해 준다.
test_clean  = test_clean.reshape(-1, img_height, img_width, 1)/255.    # 테스트 data도 동일하게 변형한다.

mu = 0
sigma = 0.1
train_noisy = train_clean + np.random.normal(mu, sigma, size=np.shape(train_clean))
test_noisy  = test_clean  + np.random.normal(mu, sigma, size=np.shape(test_clean))

In [4]:
class REDNet(tf.keras.Model):   # 딥뉴럴넷 모델 클래스 (객체)

    def __init__(self, depth=5, grayscale=True):    # 인자 depth는 네트워크의 층의 갯수에 해당하는 hyper-parameter
                                                    #
        super(REDNet, self).__init__()              # tf.keras.Model class를 상속했으므로  상속한 class의 init 함수 수행
        # Network params
        self.channels = 1 if grayscale else 3       # 인자 grayscale이 참이면 입력 채널이 1개, 컬러 영상이면 입력 채널이 3개
        self.depth = depth                          # 네트워크 층 hyper-parameter
        
        self.conv_layers = []                       # conv layer 인스턴스들을 하나의 list로 묶어서 관리
        self.deconv_layers = []                     # deconv layer 인스턴스들을 하나의 list로 묶어서 관리
        self.residual_layers = []                   # 잔차 연결에 활용한 conv layer 인스턴스들을 하나의 list로 묶어서 관리

        # 첫 번째 conv layer 인스턴스 생성 및 list에 추가
        self.conv_layers.append(tf.keras.layers.Conv2D(3, kernel_size=3, padding='same', activation='relu'))

        # Auto-Encoder 구조이므로 대응되는 deconv layer도 함께 생성/추가
        # 특징맵의 채널수는 64개를 사용
        for n in range(self.depth - 1):
            self.conv_layers.append(tf.keras.layers.Conv2D(64, kernel_size=3, padding='same', activation='relu'))
            self.deconv_layers.append(tf.keras.layers.Conv2DTranspose(64, kernel_size=3, padding='same', activation='relu'))
            
        # 마지막 deconv layer를 생성/추가
        self.deconv_layers.append(tf.keras.layers.Conv2DTranspose(self.channels, kernel_size=3, padding='same'))

    # 생성한 layer 인스턴스들을 가져와서 입력 처리
    def call(self, input_tensor):
        x = self.conv_layers[0](input_tensor)   # 생성한 네트워크 층을 불러와 input tensor를 통과시킨다.

        # 인코더
        for n in range(self.depth - 1):         # 인코더 층들을 순차적으로 통과시킨다.
            x = self.conv_layers[n + 1](x)
            if n % 2 == 0:      # 2번째 conv 층마다
                self.residual_layers.append(x)  # 잔차 연결을 생성하기 위해 저장해둔다.

        # 디코더
        for n in range(self.depth - 1):
            if n % 2 == 1:                                                    # 2번째 층마다
                x = tf.keras.layers.Add()([x, self.residual_layers.pop()])    # 잔차연결을 Add 층으로 더하여 준다
                x = tf.keras.layers.Activation('relu')(x)                     # 활성화 함수
            x = self.deconv_layers[n](x)                                      # deconv 층 통과

        return self.deconv_layers[-1](x)    # 마지막 deconv 층 출력값을 리턴

    def model(self):
        # model을 빌드하기 위한 함수
        x = tf.keras.Input(shape=(None, None, self.channels))   #
        return tf.keras.Model(inputs=[x], outputs=self.call(x)) # 네트워크 모델을 순차적으로 모두 통과한 출력값을 리턴

In [5]:
model = REDNet().model()

model.compile(optimizer='adam',     # Adam Optimizer 사용
              loss='mse',           # MSE loss 사용
              metrics=[tf.keras.metrics.MeanSquaredError()])    # 평가 시에도 MSE 사용

# 네트워크 모델의 요약 정보를 출력
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 1)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, None,      │         30 │ input_layer[0][0] │
│                     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, None,      │      1,792 │ conv2d[0][0]      │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, None,      │     36,928 │ conv2d_1[0][0]    │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, None,      │     36,928 │ conv2d_2[0][0]    │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, None,      │     36,928 │ conv2d_3[0][0]    │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, None,      │     36,928 │ conv2d_4[0][0]    │
│ (Conv2DTranspose)   │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, None,      │          0 │ conv2d_transpose… │
│                     │ None, 64)         │            │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, None,      │          0 │ add[0][0]         │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, None,      │     36,928 │ activation[0][0]  │
│ (Conv2DTranspose)   │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_2  │ (None, None,      │     36,928 │ conv2d_transpose… │
│ (Conv2DTranspose)   │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, None,      │          0 │ conv2d_transpose… │
│                     │ None, 64)         │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, None,      │          0 │ add_1[0][0]       │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_3  │ (None, None,      │     36,928 │ activation_1[0][… │
│ (Conv2DTranspose)   │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_4  │ (None, None,      │        577 │ conv2d_transpose… │
│ (Conv2DTranspose)   │ None, 1)          │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 260,895 (1019.12 KB)

 Trainable params: 260,895 (1019.12 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 테스트용 Data로 정량적 평가
test_loss, test_mse = model.evaluate(test_noisy, test_clean)

# PSNR를 출력
print('\n테스트 PSNR:', -10 * np.log10(test_mse))

# 테스트용 data에 대해 inference 수행
test_infer = model.predict(test_noisy)